# trainings

In [28]:
import yaml
import pyspark.sql.functions as F

import dextra.dna.core as C
import dextra.dna.commons as P

In [29]:
spark = P.config.spark

In [30]:
PQT = P.config.lakes.models + '/logs/encoder_trainings.parquet'

t = C.io.stream.read(PQT)
t.limit(2).toPandas()

,model_name,model_weights_path,training_proc,training,trained_at,date_received_stats,committed_at_stats,records
0,word2vec,file:///datalake/models/products/word2vec,dextra.dna.commons.processors.products.Learnin...,"{'model': {'input_col': 'text_cleaned', 'stop_...",2020-12-08 23:03:18.089839,"(2015-03-19 00:00:00, 2017-11-02 00:00:00, 146...","(2020-12-08 21:24:36.162162, 2020-12-08 21:24:...","[(861cbf751715, 2020-12-08 21:24:36.162162), (..."


### Stats from Last Training

In [32]:
last_training = (
    t.drop('records')
     .orderBy(F.desc('trained_at'))
     .limit(1)
     .collect()[0])

In [33]:
print(yaml.dump(last_training.asDict(recursive=True)))

committed_at_stats:
  avg: 1607462676.1621423
  count: 10003
  countDistinct: 1
  max: 2020-12-08 21:24:36.162162
  min: 2020-12-08 21:24:36.162162
date_received_stats:
  avg: 1468604431.790463
  count: 10003
  countDistinct: 912
  max: 2017-11-02 00:00:00
  min: 2015-03-19 00:00:00
model_name: word2vec
model_weights_path: file:///datalake/models/products/word2vec
trained_at: 2020-12-08 23:03:18.089839
training: '{''model'': {''input_col'': ''text_cleaned'', ''stop_words'': ''english'',
  ''features'': 128, ''lemmatization'': True}}'
training_proc: dextra.dna.commons.processors.products.LearningEncoder



### Encoding Example

In [37]:
x = C.io.stream.read(P.config.lakes.refined + '/issues.parquet')

In [34]:
word2vec_training = (
    t.where(t.model_name == 'word2vec')
     .orderBy(F.desc('trained_at'))
     .limit(1)
     .collect()[0])

In [38]:
from pyspark.ml import PipelineModel

encoder = PipelineModel.load(word2vec_training.model_weights_path)

In [40]:
s = x.limit(5)
s = encoder.transform(s)
s = s.toPandas()
s

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


,complaint_id,consumer_message,customer_name,date_received,disputed,issue,product,resolution,state,sub_issue,...,zip_code,ingested_at,tags_trusted_labels,tags_split,text_cleaned,committed_at,text_cleaned_lemma,text_cleaned_words,text_cleaned_filtered,features
0,0052ac552c28,Hello My name is {hash} and I have a {hash} th...,26d89fd7bd44,2017-01-30,False,Problems when you are unable to pay,Consumer Loan,Closed with explanation,GA,None,...,300XX,2020-12-08 21:22:48.137401,False,train,hello my name is {hash} and i have a {hash} th...,2020-12-08 21:24:36.162162,hello my name is {hash} and i have a {hash} th...,"[hello, my, name, is, {hash}, and, i, have, a,...","[hello, name, {hash}, {hash}, santander, consu...","[-0.046171571916112535, -0.01197570159386557, ..."
1,00a844d52aec,I have written several letters to the creditor...,89dbbe6ad0b5,2016-11-11,True,Credit reporting company's investigation,Credit reporting,Closed with explanation,OH,Investigation took too long,...,440XX,2020-12-08 21:22:48.137401,False,train,i have written several letters to the creditor...,2020-12-08 21:24:36.162162,i have written several letters to the creditor...,"[i, have, written, several, letters, to, the, ...","[written, several, letters, creditor, {hash}, ...","[-0.035483965111014086, 0.060496491012408546, ..."
2,00ae385f4af9,This agency is reporting an account on my redi...,321a61586171,2016-10-27,False,Other,Credit card,Closed with explanation,IL,None,...,604XX,2020-12-08 21:22:48.137401,False,train,this agency is reporting an account on my redi...,2020-12-08 21:24:36.162162,this agency is reporting an account on my redi...,"[this, agency, is, reporting, an, account, on,...","[agency, reporting, account, redit, profile, b...","[-0.12961782080431777, -0.008926396335785586, ..."
3,00da7e528dac,In {hash}/{hash}/{hash} or {hash}/{hash}/{hash...,f11c4528f211,2017-08-30,None,Fraud or scam,"Money transfer, virtual currency, or money ser...",Untimely response,PA,None,...,160XX,2020-12-08 21:22:48.137401,False,test,in {hash} {hash} {hash} or {hash} {hash} {hash...,2020-12-08 21:24:36.162162,in {hash} {hash} {hash} or {hash} {hash} {hash...,"[in, {hash}, {hash}, {hash}, or, {hash}, {hash...","[{hash}, {hash}, {hash}, {hash}, {hash}, {hash...","[-9.312533341299474e-06, -0.03235167523448849,..."
4,010554da4e3a,"In my previous complaint Equifax states "" Equi...",3fb5aa9ff928,2016-01-06,False,Incorrect information on credit report,Credit reporting,Closed with explanation,CA,Reinserted previously deleted info,...,913XX,2020-12-08 21:22:48.137401,False,test,in my previous complaint equifax states equifa...,2020-12-08 21:24:36.162162,in my previous complaint equifax states equifa...,"[in, my, previous, complaint, equifax, states,...","[previous, complaint, equifax, states, equifax...","[-0.02981079175568572, -0.004501622056633961, ..."


In [47]:
import numpy as np

In [59]:
s.text_cleaned[0][:LETTERS], s.text_cleaned_lemma[0][:LETTERS]

('hello my name is {hash} and i have a {hash} through santander consumer usa i got the truck back in {',
 'hello my name is {hash} and i have a {hash} through santander consumer usar i got the truck back in ')

In [58]:
LETTERS = 100
s.text_cleaned[0][:LETTERS] == s.text_cleaned_lemma[0][:LETTERS]

False